# Visão Computacional em Python utilizando as bibliotecas Scikit-Image e Scikit-Learn 

### Algoritmos de visão computacional tem como entrada matrizes bidimensionais ou multidimensionais e produzem uma informação simples como saída.

**Etapas do PDI**

* Segmentação
* Extração
* Classificação

**Áreas**

* Visão Computacional: entra imagem, sai dado.
* Computação Gráfica: entra dado, sai imagem.
* PDI: Entra imagem, sai imagem.
***

# Segmentação

* Agrupamento de pixels em uma componente conexa relevante.
* Subdivisões da imagem em regiões ou objetos segundo um critério.
* Divide uma região espacial R em N subregiões.
* Limiarização 
* Etapa Fundamental: Implementação de algoritmos para o reconhecimento das diferentes regiões da imagem através das características dos pixels ou distribuição deles na imagem.
* Calculo do valor ótimo de limiar


## Código para segmentação em python usando Scikit-image

### Importando Bibliotecas e Funções

In [1]:
from skimage.io import imread, imsave #funcao do pacote skimage para leitura de imagens, recebe o endereço da imagem como parâmetro
from skimage.color import rgb2grey #transforma uma magem RGB(colorida) em níveis de cinza, recebe imagem como parâmetro
from skimage.filters import threshold_otsu #funcao que implementa o limiar de Otsu, imagem em tons de cinza como parâmetro
import numpy as np #pacote python que oferece suporte a arrays e matrizes multidimensionais
from skimage.measure import label, regionprops #label: Detecta pequenas regiões que não fazem parte da região de interesse
                                               #regionprops: cálculo de propriedades importantes em cada região encontrada

### Carregando imagem, convertendo para níveis de cinza e calculando o limiar usando otsu

In [2]:
path_image = "databases-master/database2/imagem (1).png" #variavel recebe o endereço da imagem
imagem = imread(path_image) #Leitura da imagem
grey_image = rgb2grey(imagem) #Convertendo a imagem de colorido para tons de cinza
otsu = threshold_otsu(grey_image) #Realizando o calculo do limiar usando otsu
img_otsu = grey_image < otsu #Pixels com valores menores que 'otsu' serão brancos, caso contrario serao pretos

### Remoção de pequenas regiões

In [3]:
#Detecção e cálculo de propriedades
label_img = label(img_otsu, connectivity = grey_image.ndim) #Detecta regiões não conectadas
props = regionprops(label_img) #Calcula propriedades importantes de cada região encontrada (ex.: area)

#Converte todas as regiões que possuem um valor de área menor que a maior area em background da imagem
area = np.asarray([props[i].area for i in range(len(props))]) #Área de cada região encontrada
max_index = np.argmax(area) #Index da maior região

for i in range(len(props)):
    if (props[i].area < props[max_index].area):
        label_img[np.where(label_img == i+1)] = 0 #Região menor que a maior é marcada como background        

label_img = label_img*1 #Transforma imagem de valores booleanos para inteiros

### Recorte da região de interesse

In [4]:
#Obtendo os limites verticais das imagens segmentadas
ymin = np.min(np.where(label_img == True)[1])
ymax = np.max(np.where(label_img == True)[1])
imagem_segmentada = imagem[:,ymin:ymax,:]
imsave("imagemseg.png", imagem_segmentada)

***
# Extração de Atributos

* As características (propriedades) dos objetos na imagem podem ser quantificadas e usadas como descritores da imagem ou de objetos em uma cena.
* O objeto pode ser representado por um espaço R^n, definido em termos das N características
* Invariância a transformações afins: invariante a rotação, escala e translação (Propriedade desejável a descritores).
* Modos de representar regiões (Passo essencial):
    * Através de características externas, pertencentes a fronteira.
    * Em termos de características internas, pixels que compõem a região.
* Descrever a região com base na representação escolhida.
* Informações adquiridas na extração de atributos formam um vetor de atributos.

## Matriz de Coocorrência em Níveis de Cinza (GLCM)

* Técnica que tem como base a análise de textura em imagem.
* Análise de coocorrências existentes entre pares de pixels através de algum padrão.
* Sempre quadrada, guarda informações das intensidades relativas dos pixels.
* As coocorrências são calculadas para determinadas distâncias ou espaçamento entre pares de pixels e uma orientação.
* Existe uma matriz para cada relacionamento entre distância e orientação.
* De acordo com Haralick, são 14 características significativas para GLCM e outras medidas derivadas foram acrescentadas.
    * A utilização de apenas um subconjunto dessas características pode (em alguns casos) gerar melhor desempenho do que a utilização de todas.

## Código para a extração de atributos em Python usando Scikit-Image

* Realizada em duas etapas:
    * Cálculo da matriz de coocorrências 
    * Cálculo dos atributos de Haralick

### Importando Bibliotecas e Funções

In [5]:
from skimage.feature import greycomatrix, greycoprops
from skimage import img_as_ubyte

#greycomatrix: realiza o cálculo da matriz. Parâmetros: imagem, distância, ângulo, normed (se true, retorna matriz normalizada)
#greycoprops: calcula os atributos de Haralick. Parâmetros: matriz calculada e o atributo a ser calculado.
#image_as_ubyte: conversão de valores na matriz da imagem. Converte para valores unsigned integer entra 0 e 255. Parâmetros: matriz da imagem a ser convertida.

### Cálculo da matriz GLCM e dos Atributos Haralick

In [6]:
image_path = "imagemseg.png" #Endereço da imagem
image = imread(image_path) #Leitura da imagem
image_grey = rgb2grey(imagem) #Convertendo de RGB para tons de cinza
img = img_as_ubyte(image_grey)
d = 1 #Distância entre pixels GLCM
matrix = greycomatrix(img, [d], [0], normed = True) #Cálculo da matriz em 0 graus

/home/acucena/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [7]:
props = np.zeros((6)) #Vetor para armazenar atributos
props[0] = greycoprops(matrix, 'contrast') #Calcula contrast 
props[1] = greycoprops(matrix, 'dissimilarity') #Calcula dissimilarity 
props[2] = greycoprops(matrix, 'homogeneity') #Calcula homogeneity 
props[3] = greycoprops(matrix, 'energy') #Calcula energy 
props[4] = greycoprops(matrix, 'correlation') #Calcula correlation 
props[5] = greycoprops(matrix, 'ASM') #Calcula segundo momento angular 

***
# Aprendizagem de Máquina e Classificação de Imagens

* Reconhecimento de padrões: classificação de determinado objeto ou situação como pertencente a um certo número de categorias.
* Algoritmos de aprendizado de máquina supervisionados: algoritmos em que é preciso apresentar dados pré-classificados e "ensinar" o algoritmo a identificar diferentes objetos.
    * Necessário que ao menos parte dos dados sejam previamente avaliados e rotulados por especialistas.
    * Ex.: SVM, MLP, RF

## Máquina de Vetor de Suporte - SVM

* Faz um mapeamento do espaço de entrada para um espaço de dimensionalidade maior.
* Calcula um hiperplano de separação ótimo.
    * Escolhido de modo a maximizar a distância de separação entre as classes.
    * Duas classes são linearmente separáveis se existe um hiperplano divisório entre as amostras de classes diferentes.
    * O que é um hiperplano?
        * Trata-se de uma linha que ao ser traçada consegue separar duas classes distintas em cada parte do plano cartesiano.
    * O que é o hiperplano de separação ideal?
        * É um hiperplano que é tão longe quanto possível dos pontos de cada categoria de forma que ele classifica os dados de maneira correta e generaliza melhor com dados não usados no treinamento.
* Trata-se de um algoritmo supervisionado: possui fases de treinamento e teste.

* Vetores de suporte
    * Projetados no treinamento.
    * Usados para encontrar um hiperplano de separação ótimo.
* Como funciona
    * Aumenta a dimensionalidade da entrada para transformar funções não-linearmente separáveis em funções linearmente separáveis.
    * Funções de Kernel:
        * Utilizadas para aumentar a dimensão de um vetor de entrada.
        * Aumento de uma dimensão N para uma X, sendo X > N.
    * Após a realização dos cálculos, são obtidos os vetores de suporte e a partir deles é obtido o hiperplano de separação ótimo, onde cada lado do mesmo representa uma classe.

## Random Forest

* Combinação de predições de diversas árvores em que cada árvore depende dos valores de um vetor independente, amostrados aleatoriamente e com a mesma distribuição para cada árvore da floresta.
    * Floresta: coleção de árvores de decisão.
* Flexível e fácil de usar(Dois parâmetros: nº de variáveis do subconjunto aleatório em cada nó e o número de árvores da floresta). Produz excelentes resultados, na maioria das vezes.
* Como funciona?
    * Algoritmo de aprendizagem supervisionado.
    * Cria uma floresta de modo aleatório.
        * Combinação de árvores de decisão, em sua maioria, treinadas com o método bagging.
        * Método bagging: combinação dos modelos de aprendizado aumenta o resultado geral.
    * Resumindo: cria várias árvores de decisão e as combina para obter uma predição com maior acurácia e mais estável.
    * Adiciona aleatoriedade extra ao modelo quando está criando as árvores
        * Procuram a melhor característica ao dividir os nós.
        * As características se subdividem aleatoriamente em subconjuntos(vetores) de características.
        * Busca a melhor característica dentro dos subconjuntos. Para cada árvore é gerado um subconjunto de características.
    * Escolhe as classes com o maior número de votos na floresta.

## Código para classificação de imagens usando SVM e RF em Python usando Scikit-Learn 

* Scikit-learn: Funções de aprendizagem de máquina, clusterização, seleção de atributos e outras.
* Etapas básicas da classificação:
    * Divisão do conjunto de dados em treino e teste.
    * Criação de uma instância do classificador utilizado.
    * Treino do classificador.
    * Predição utilizando casos de teste.
    * Cálculo da taxa de acertos seguindo alguma métrica de avaliação de desempenho.

### Importando Bibliotecas e funções

In [ ]:
from sklearn.svm import SVC #SVM para classificação
from sklearn.ensemble import RandomForestClassifier #RF para classificação
from sklearn.model_selection import train_test_split #Pacote para dividir os dados em treino e teste
from sklearn.metrics import accuracy_score #Métrica de Avaliação

### Divisão do conjunto de dados em treino e teste

In [ ]:
#Não executar, não funciona pois é um exemplo de como deve ser aplicada a função e não a aplicação em si.
train = 0.3 #30% dos dados para treino
test = 1-train #Resto dos dados(70%) para testar o classificador 
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=test) #Divisão dos dados em conjuntos de treino e teste

### Instância, treino, teste e cálculo de acertos usando SVM 

In [ ]:
#Não executar, não funciona pois é um exemplo de como deve ser aplicada a função e não a aplicação em si.
cfl_svm = SVC() #Instância da SVM
clf_svm.fit(X_train, Y_train) #Treinamento da SVM
pred_svm = clf_svm(X_test) #Teste da SVM
acc_svm = accuracy_score(Y_test, pred_svm) #Cálculo de acertos

### Instância, treino, teste e cálculo de acertos usando Random Forest

In [ ]:
#Não executar, não funciona pois é um exemplo de como deve ser aplicada a função e não a aplicação em si.
clf_rf = RandomForestClassifier() #Cria uma instância da Random Forest
clf_rf.fit(X_train, Y_train) #Treinamento da Random Forest
pred_rf = clf_rf.predict(X_test) #Teste da RF
acc_rf = acurracy_score(Y_test, pred_rf) #Cálculo de acertos

## Análise do Componentes Principais (PCA)

* Técnica que usa princípios da álgebra linear para transformar variáveis, possivelmente correlacionadas, em um número menor de variáveis chamadas de componentes principais.
* Busca reduzir o número de dimensões em um set de dados. Projetando os dados em um novo plano.
* Com essa nova projeção, os dados originais podem ser interpretado usando menos dimensões.
* Com dados reduzidos pode-se observar com mais clareza tendências, padrões e outliers.
* Fornece apenas clareza a padrões que já estão lá.
* Objetivo: Diminuir a quatidade de atributos mantendo a acurácia.
* Como funciona?
    * Realiza o cálculo da média da posição dos pontos
    * Com a média calculada, calcula a matriz de covariância.
    * Após a matriz calculada, sendo ela real e simétrica, pode-se encontrar n autovetores (vetor que ao ser multiplicado por uma matriz retorna um valor e ao ser multiplicado por um autovalor retorna o mesmo valor da multiplicação pela matriz)
    * Com esses autovetores forma-se uma nova matriz, ordenada pelo autovalores de cada vetor (do maior para o menor). A diagonal principal é formada pelos autovalores de cada autovetor.

## Código para calculo das componentes principais em Python usando Scikit-Learn 

In [ ]:
#Não executar, não funciona pois é um exemplo de como deve ser aplicada a função e não a aplicação em si.
from sklearn.decomposition import PCA #Biblioteca que implementa a análise de componentes principais

components = [2, 4, 8, 10, 12] #Quantidade de componentes desejadas, esses valores devem ser menores que a quantidade de atributos.
#Aqui extraímos 6 atributos de cada componente no sistema de cores RGB, totalizando 18 atributos.

#Função que calcula PCA
def pca(X_train, X_test, Y_train, n_comp):
    
    pca = PCA(n_components=ncomp)
    pca.fit(X_train, Y_train)
    transform = pca.transform(X_test)
    return transform

results = np.zeros(5) #Variável que armazena os resultados para cada componente

#Para cada quantidade de componentes o PCA é executado nos conjuntos de treino e teste. Após isso os dados são classificados.
for id_comp, comp in enumerate(components):
    X_train_pca = pca(X_train, X_test, Y_train, comp)
    X_test_pca = pca(X_train, X_test, Y_train, comp)
    c_rf = RandomForestClassifier()
    c_rf.fit(X_train_pca, Y_train)
    pred = c_rf.predict(X_test_pca)
    acc = accuracy_score(y_test, pred)
    results[id_comp] = acc

# Código-Fonte 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread_collection, imsave
from sklearn.model_selection import train_test_split
from glob import glob
from scipy.stats import randint as sp_randint
from skimage.color import rgb2grey
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from sklearn.decomposition import PCA
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.svm import SVC
import time

In [3]:
path = "databases-master/database2_raw_data"

comcapacete = glob(path+'comcapacete/*.png')
semcapacete = glob(path+'semcapacete/*.png')

images_cc = imread_collection(comcapacete)
images_sc = imread_collection(semcapacete)

In [4]:
def segmentation(im):
    '''Recebe uma imagem calcula limiar de otsu e fazer o 
       recorte obdecendo a região resultante desse limiar'''
    
    grey_image = rgb2grey(im)
    otsu = threshold_otsu(grey_image)
    im_ = grey_image < otsu 
    
    n_branco = np.sum(im_ == 1)
    n_preto = np.sum(im_ == 0)
    if n_branco > n_preto:
        im_ = 1-im_
    
    label_img = label(im_, connectivity = grey_image.ndim)#detecta regioes nao conectadas
    props = regionprops(label_img)#calcula propriedade importantes de cada regiao encontrada (ex. area)

    #Convert todas as regioes que possuem um valor de area menor que a maior area em background da imagem 
    area = np.asarray([props[i].area for i in range(len(props))])#area de cada regiao encontrada
    max_index = np.argmax(area)#index da maior regiao
    for i in range(len(props)):
        if(props[i].area < props[max_index].area):
            label_img[np.where(label_img == i+1)] = 0#regiao menor que a maior eh marcada como background
 
    #----------------recorte da regiao de interesse----------------#
    # Obtendo os limites verticais das imagens segmentadas 
    ymin = np.min(np.where(label_img != 0)[1])
    ymax = np.max(np.where(label_img != 0)[1])
    imagem_cortada = imagem[:,ymin:ymax,:]     
    return imagem_cortada

In [7]:
start = time.time()
for id_im,imagem in enumerate(images_cc):
    im_name = images_cc.files[id_im].split('/')[-1]
    imagem_segmentada = segmentation(imagem)
    imsave(path+'segmentacao/comcapacete/'+im_name,imagem_segmentada)

    print(im_name)
for id_im,imagem in enumerate(images_sc):
    im_name = images_sc.files[id_im].split('/')[-1]
    imagem_segmentada = segmentation(imagem)
    imsave(path+'segmentacao/semcapacete/'+im_name,imagem_segmentada)
    print(im_name)
    
end = time.time()

print("Tempo para segmentação das imagens: ", end - start)

Tempo para segmentação das imagens:  0.0003325939178466797


In [8]:
labels = np.concatenate((np.zeros(len(comcapacete)),np.ones(len(semcapacete))))

## Extração de características usando GLCM

In [9]:
path_segmentada = "databases-master/database2_raw_data/segmentacao/"
comcapacete = glob(path_segmentada+'comcapacete/*.png')
semcapacete = glob(path_segmentada+'semcapacete/*.png')
images = imread_collection(comcapacete+semcapacete)

In [10]:
d = 15

features = np.zeros((len(labels),18)) #6 features x 3 color channels

start = time.time()

for id_im,imagem in enumerate(images):
    for id_ch in range(3):
        matrix0 = greycomatrix(imagem[:,:,id_ch], [d], [0],normed=True)
        matrix1 = greycomatrix(imagem[:,:,id_ch], [d], [np.pi/4],normed=True)
        matrix2 = greycomatrix(imagem[:,:,id_ch], [d], [np.pi/2],normed=True)
        matrix3 = greycomatrix(imagem[:,:,id_ch], [d], [3*np.pi/4],normed=True)
        matrix = (matrix0+matrix1+matrix2+matrix3)/4 
        props = np.zeros((6))
        props[0] = greycoprops(matrix,'contrast')
        props[1] = greycoprops(matrix,'dissimilarity')
        props[2] = greycoprops(matrix,'homogeneity')
        props[3] = greycoprops(matrix,'energy')
        props[4] = greycoprops(matrix,'correlation')
        props[5] = greycoprops(matrix,'ASM')
        features[id_im,id_ch*6:(id_ch+1)*6] = props

end = time.time()

print(end - start)

IndexError: index 0 is out of bounds for axis 0 with size 0

## Divisão de Dados

In [9]:
train = 0.5
test = 1-train
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test)

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer, confusion_matrix,accuracy_score
from sklearn.svm import SVC

#Random Forest Parameter Estimation
def rf_parameter_estimation(xEst, yEst):
    
    clf = RandomForestClassifier(n_estimators=20)    
    # specify parameters and distributions to sample from
    hyperparameters = {"n_estimators": range(10,1000,50),
                  "max_depth": range(1,100),
                  "max_features": sp_randint(1, xEst.shape[1]),
                  "min_samples_split": sp_randint(1, xEst.shape[1]),
                  "min_samples_leaf": sp_randint(1, xEst.shape[1]),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    
    
    # run randomized search
    n_iter_search = 20
    random_search = RandomizedSearchCV(clf, param_distributions=hyperparameters,
                                       n_iter=n_iter_search,scoring=make_scorer(accuracy_score))
    
    
    random_search.fit(xEst, yEst)
    report(random_search.cv_results_)
    return random_search.best_params_

#SVM Parameter Estimation
def svm_parameter_estimation(xEst, yEst):
    
    hyperparameters = {'gamma': [1e-1, 1e-2,1e-3, 1e-4],'C': [1, 10, 100, 1000]}
    
    clf = SVC(kernel='rbf')
    # Run randomized search
    n_iter_search = 8
    random_search = RandomizedSearchCV(clf, param_distributions=hyperparameters,
                                       n_iter=n_iter_search,scoring=make_scorer(accuracy_score))    
    random_search.fit(xEst, yEst)

    report(random_search.cv_results_)
    return random_search.best_params_

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))

/home/hidel/anaconda3/envs/AcucenaICV/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Classificação

In [10]:
start = time.time()

parameters = rf_parameter_estimation(X_train, y_train)
c_rf = RandomForestClassifier(**parameters)
c_rf.fit(X_train,y_train)
pred = c_rf.predict(X_test)
acc_rf = accuracy_score(y_test, pred)

end = time.time()
print(end - start)

print('Random Forest Accuracy: ',acc_rf)

start = time.time()

parameters = svm_parameter_estimation(X_train, y_train)
c_svm = SVC(**parameters)
c_svm.fit(X_train,y_train)
pred = c_svm.predict(X_test)
acc_svm = accuracy_score(y_test, pred)

end = time.time()
print(end - start)

parameters

print('Support Vector Machine Accuracy: ',acc_svm)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.